# Utility Traits 

## Drop

In [2]:
:dep libc

In [3]:
//use std::os::unix::io::{RawFd, FromRawFd, AsRawFd};
//use std::fs::File;
use std::io::{self};
use std::os::raw::c_int;
use std::str;

static mut FD: i32 = 42;

#[derive(Debug)]
pub struct FileDescriptor {
    fd: c_int,
    file_name: String
}

impl FileDescriptor {
    // Create a new FileDescriptor from a RawFd
    pub fn new(path: &str) -> Self {
        // real implementation under unix
        // let fd = unsafe { libc::open(path.as_ptr() as *const i8, libc::O_RDWR) };
        // FileDescriptor { fd }

        let fd: FileDescriptor;
        unsafe {
            FD += 1;
            fd = FileDescriptor{ fd: FD, file_name: path.to_string() };
        }
        println!("Opening file - {:?}", fd);
        fd
    }

    // Write to the file descriptor
    pub fn write(&self, buf: &[u8]) -> io::Result<usize> {
        // real implementation
        // let mut file = unsafe { File::from_raw_fd(self.fd) };
        // let result = file.write(buf);
        // std::mem::forget(file); // Prevent the file from being closed
        // result

        println!(r###"Writing "{}" to file {:?}"###, str::from_utf8(buf).unwrap(), self);
        return Ok(buf.len());
    }
}

impl Drop for FileDescriptor {
    fn drop(&mut self) {
        // real implementation
        // unsafe {
        //     libc::close(self.fd);
        // }

        println!("Closing file - {:?}", self);
    }
}

fn main() -> io::Result<()> {
    let mut fd1 = FileDescriptor::new("file_descriptor_test1.txt"); // 1 is the file descriptor for stdout
    {
        fd1.write(b"Hello, world!")?;
        
        let fd2 = FileDescriptor::new("file_descriptor_test2.txt");
        fd2.write("Rust is awesome!".as_bytes())?;

        fd1 = fd2;
    }

    fd1.write("Goodbye, world!".as_bytes())?;

    Ok(())
}

main()

Opening file - FileDescriptor { fd: 43, file_name: "file_descriptor_test1.txt" }
Writing "Hello, world!" to file FileDescriptor { fd: 43, file_name: "file_descriptor_test1.txt" }
Opening file - FileDescriptor { fd: 44, file_name: "file_descriptor_test2.txt" }
Writing "Rust is awesome!" to file FileDescriptor { fd: 44, file_name: "file_descriptor_test2.txt" }
Closing file - FileDescriptor { fd: 43, file_name: "file_descriptor_test1.txt" }
Writing "Goodbye, world!" to file FileDescriptor { fd: 44, file_name: "file_descriptor_test2.txt" }
Closing file - FileDescriptor { fd: 44, file_name: "file_descriptor_test2.txt" }


Ok(())

## Sized Trait

* A sized type is one whose values all have the same size in memory. 
  * Almost all types in Rust are sized:
    * `u64` takes eight bytes
    * `(f32, f32, f32)` tuple takes twelve bytes 
    * Structs & Enums are sized
    * `String` and `Vec<T>` own a heap-allocated buffer whose size can vary, the `Vec` value itself is a pointer to the buffer, its capacity, and its length, so `Vec<T>` is a sized type.
* All sized types implement the `std::marker::Sized` trait, which has no methods or associated types. 
  * Rust implements it automatically for all types to which it applies; you can’t implement it yourself. 
* The only use for `Sized` is as a bound for type variables: a bound like `T: Sized` requires `T` to be a type whose size is known at compile time. Traits of this sort are called *marker traits*, because the Rust language itself uses them to mark certain types as having characteristics of interest.

* However, Rust also has a few unsized types whose values are not all the same size. 
  * The string slice type `str` (without an `&`) is unsized. The string literals `"diminutive"` and `"big"` are references to str slices that occupy ten and three bytes. 
  * Array slice types like `[T]` (again, without an `&`) are unsized. A shared reference like `&[u8]` can point to a `[u8]` slice of any size. Because the `str` and `[T]` types denote sets of values of varying sizes, they are unsized types.
  * `dyn Type` is unsized - but `&dyn std::io::Write` and `Box<dyn std::io::Write>` are fat pointers (two words wide: a pointer to a slice also carries the slice’s length, and a trait object also carries a pointer to a vtable of method implementations) and are sized

## Clone

* The `std::clone::Clone` trait is for types that can make copies of themselves. `Clone` is defined as follows:

In [4]:
mod explain {
    trait Clone: Sized {
        fn clone(&self) -> Self;
        fn clone_from(&mut self, source: &Self) { 
            *self = source.clone(); 
        }
    }
}

In [5]:
#[derive(Debug, Clone, PartialEq)]
struct Data {
    name: String,
    id: i32,
    data: Vec<i32>,
}

let user_data = Data { name: "John".to_string(), id: 42, data: vec![1, 2, 3] };
let mut backup_data = user_data.clone();

assert_eq!(user_data, backup_data);

backup_data.name = "Jane".to_string();

assert_ne!(user_data, backup_data);

## Copy

* A type is `Copy` if it implements the `std::marker::Copy` marker trait, which is defined as follows:

In [6]:
mod explain {
    trait Copy: Clone { }
}

In [7]:
#[derive(Clone)]
struct ValueData {
    id: u64,
    data: f64
}

impl Copy for ValueData { }

## Deref & DerefMut

* You can specify how dereferencing operators like `*` and `.` behave on your types by implementing the `std::ops::Deref` and `std::ops::DerefMut` traits. 
* Pointer types like `Box<T>` and `Rc<T>` implement these traits so that they can behave as Rust’s built-in pointer types do.

In [8]:
:dep num-complex

use num_complex::Complex;

let z: Box<Complex<f64>> = Box::new(Complex{re: 1.0, im: 2.0});

assert_eq!((*z).re, 1.0);
assert_eq!(z.re, 1.0); // deref coercion

The type of the variable user_data was redefined, so was lost.
The type of the variable backup_data was redefined, so was lost.


In [9]:
mod explain {
    trait Deref {
        type Target: ?Sized;
        fn deref(&self) -> &Self::Target;
    }
    
    trait DerefMut: Deref {
        fn deref_mut(&mut self) -> &mut Self::Target;
    }
}

* The `Deref` and `DerefMut` are used for **deref coercions**. 
  * Since deref takes a `&Self` reference and returns a `&Self::Target` reference, Rust uses this to **automatically convert references of the former type into the latter**. 
  * In other words, if inserting a deref call would prevent a type mismatch, Rust inserts one for you. Implementing `DerefMut` enables the corresponding conversion for mutable references. These are called the **deref coercions: one type is being “coerced” into behaving as another**.

In [10]:
struct Selector<T> {
    /// Elements available in this `Selector`.
    elements: Vec<T>,

    /// The index of the "current" element in `elements`. A `Selector`
    /// behaves like a pointer to the current element.
    current: usize
}

use std::ops::{Deref, DerefMut};

impl<T> Deref for Selector<T> {
    type Target = T;
    fn deref(&self) -> &T {
        &self.elements[self.current]
    }
}

impl<T> DerefMut for Selector<T> {
    fn deref_mut(&mut self) -> &mut T {
        &mut self.elements[self.current]
    }
}

In [11]:
let mut s = Selector { elements: vec!['x', 'y', 'z'], current: 2 };

// Because `Selector` implements `Deref`, we can use the `*` operator to
// refer to its current element.
assert_eq!(*s, 'z');

// Assert that 'z' is alphabetic, using a method of `char` directly on a
// `Selector`, via deref coercion.
assert!(s.is_alphabetic());

// Change the 'z' to a 'w', by assigning to the `Selector`'s referent.
*s = 'w';

assert_eq!(s.elements, ['x', 'y', 'w']);

In [12]:
let s = Selector { 
    elements: vec!["good", "bad", "ugly"],
    current: 2 
};

fn show_it(thing: &str) { 
    println!("{}", thing); 
}

show_it(&s); // OK because `&Selector<T>` can be coerced to `&T`

ugly


* Rust do not apply coercion when checking whether the bound `T: Trait` is satisfied

In [13]:
use std::fmt::Display;

fn show_it_generic<T: Display>(thing: T) { println!("{}", thing); }

//show_it_generic(&s); // error[E0277]: the trait bound `&Selector<&str>: Display` is not satisfied

In [14]:
show_it_generic(&s as &str); // OK

ugly


## Default

* Some types have a reasonably obvious default value: 
  * the default vector or string is empty, 
  * the default number is zero, 
  * the default `Option` is `None`, and so on. 
* Types like this can implement the `std::default::Default` trait:

In [15]:
mod explain {
    trait Default {
        fn default() -> Self;
    }
}

* If a type `T` implements `Default`, then the standard library implements `Default` automatically for `Rc<T>`, `Arc<T>`, `Box<T>`, `Cell<T>`, `RefCell<T>`, `Cow<T>`, `Mutex<T>`, and `RwLock<T>`. The default value for the type `Rc<T>`, for example, is an `Rc` pointing to the default value for type `T`.

* If all the element types of a tuple type implement `Default`, then the tuple type does too, defaulting to a tuple holding each element’s default value.

* Rust does not implicitly implement `Default` for struct types, but if all of a struct’s fields implement `Default`, you can implement `Default` for the struct automatically using `#[derive(Default)]`.

* Usage of common use of `Default` is to produce default values for structs that represent a large collection of parameters, most of which you won’t usually need to change

In [16]:
#[derive(Default)]
struct Player {
    name: String,
    age: u8,
    level: u8,
    score: u32,
    record_score: u32
}

let player = Player { 
    name: "John".to_string(),
    age: 42,
    ..Default::default()
};

## AsRef and AsMut

* When a type implements `AsRef<T>`, that means you can borrow a `&T` from it efficiently. `AsMut` is the analogue for mutable references. 

In [17]:
mod explain {
    trait AsRef<T: ?Sized> {
        fn as_ref(&self) -> &T;
    }
    
    trait AsMut<T: ?Sized> {
        fn as_mut(&mut self) -> &mut T;
    }
}

* For example:
  * `Vec<T>` implements `AsRef<[T]>` 
  * `String` implements `AsRef<str>` and you can also borrow a `String`’s contents as an array of bytes, so `String` implements `AsRef<[u8]>` as well.

* `AsRef` is typically used to make functions more flexible in the argument types they accept. For example, the `std::fs::File::open` function is declared like this:

In [58]:
use std::fmt::Debug;

#[derive(Debug)]
struct SomeData {
    values: Vec<i32>
}

impl AsRef<[i32]> for SomeData {
    fn as_ref(&self) -> &[i32] {
        &self.values
    }
}

fn process_data<T: AsRef<[i32]> + Debug>(data: T) {
    println!("Processing data: {:?}", data);
}

let data = SomeData { values: vec![1, 2, 3] };

process_data(&data);

Processing data: SomeData { values: [1, 2, 3] }


## Borrow and BorrowMut

* The `std::borrow::Borrow` trait is similar to `AsRef`: if a type implements `Borrow<T>`, then its borrow method efficiently borrows a `&T` from it. 
* But `Borrow` imposes more restrictions: a type should implement `Borrow<T>` only when a `&T` hashes and compares the same way as the value it’s borrowed from. (Rust doesn’t enforce this; it’s just the documented intent of the trait.) This makes Borrow valuable in dealing with keys in hash tables and trees or when dealing with values that will be hashed or compared for some other reason.
  * `String` implements `AsRef<str>`, `AsRef<[u8]>`, and `AsRef<Path>`, but those three target types will generally have different hash values. Only the `&str` slice is guaranteed to hash like the equivalent `String`, so `String` implements only `Borrow<str>`.

In [18]:
mod explain {
    trait Borrow<Borrowed: ?Sized> {
        fn borrow(&self) -> &Borrowed;
    }
}

## From and Into

* The `std::convert::From` and `std::convert::Into` traits represent conversions that consume a value of one type and return a value of another. 
* Whereas the `AsRef` and `AsMut` traits borrow a reference of one type from another, `From` and `Into` take ownership of their argument, transform it, and then return ownership of the result back to the caller.

In [19]:
mod explain {
    trait From<T>: Sized {
        fn from(other: T) -> Self;
    }

    trait Into<T>: Sized {
        fn into(self) -> T;
    }
}

* The standard library automatically implements the trivial conversion from each type to itself: every type `T` implements `From<T>` and `Into<T>`

* Although the traits simply provide two ways to do the same thing, they lend themselves to different uses.

### Using Into
* You generally use `Into` to make your functions more flexible in the arguments they accept. For example, if you write:

In [20]:
use std::net::Ipv4Addr;
fn ping<A>(address: A) -> std::io::Result<bool>
    where A: Into<Ipv4Addr>
{
    let ipv4_address = address.into();
    // rest implementation
    println!("Pinging address: {}", ipv4_address);
    Ok(true)
}

println!("{:?}", ping(Ipv4Addr::new(23, 21, 68, 141))); // pass an Ipv4Addr
println!("{:?}", ping([66, 146, 219, 98]));             // pass a [u8; 4]
println!("{:?}", ping(0xd076eb94_u32));                 // pass a u32

Pinging address: 23.21.68.141
Ok(true)
Pinging address: 66.146.219.98
Ok(true)
Pinging address: 208.118.235.148
Ok(true)


### Using From

 * The `from` method serves as a generic constructor for producing an instance of a type from some other single value. 
 * For example, rather than `Ipv4Addr` having two methods named `from_array` and `from_u32`, it simply implements `From<[u8;4]>` and `From<u32>`, allowing us to write:

In [21]:
let addr1 = Ipv4Addr::from([66, 146, 219, 98]);
let addr2 = Ipv4Addr::from(0xd076eb94_u32);

* Given an appropriate `From` implementation, the standard library automatically implements the corresponding `Into` trait. 
* When you define your own type, if it has single-argument constructors, you should write them as implementations of `From<T>` for the appropriate types; you’ll get the corresponding `Into` implementations for free.


* `from` and `into` methods take ownership of their arguments (conversion can reuse the original values)

In [22]:
let text = "Hello Rusty!".to_string();
let bytes: Vec<u8> = text.into();
println!("{:?}", bytes);

[72, 101, 108, 108, 111, 32, 82, 117, 115, 116, 121, 33]


* `From` and `Into` are infallible traits—their API requires that conversions will not fail

In [24]:
let huge = 2_000_000_000_000i64;
let smaller: i32 = huge as i32;
println!("{:?}", smaller);

-1454759936


## TryFrom & TryInto

* `TryFrom` and `TryInto` are the fallible cousins of `From` and `Into` and are similarly reciprocal; implementing `TryFrom` means that `TryInto` is implemented as well.

In [ ]:
mod explain {
    pub trait TryFrom<T>: Sized {
        type Error;
        fn try_from(value: T) -> Result<Self, Self::Error>;
    }

    pub trait TryInto<T>: Sized {
        type Error;
        fn try_into(self) -> Result<T, Self::Error>;
    }
}

In [26]:
let smaller = huge.try_into().unwrap_or(i32::MAX);
smaller

2147483647

In [28]:
let smaller: i32= huge.try_into().unwrap_or_else(|_| {
    if huge >= 0 {
        i32::MAX
    } else {
        i32::MIN
    }
});
smaller

2147483647

## ToOwned

* `The std::borrow::ToOwned` trait provides a slightly looser (than `Clone` trait) way to convert a reference to an owned value:

In [30]:
mod explain {
    trait ToOwned {
        type Owned: std::borrow::Borrow<Self>;
        fn to_owned(&self) -> Self::Owned;
    }
}

* Unlike `clone`, which must return exactly `Self`, `to_owned` can return anything you could borrow a `&Self` from: the `Owned` type must implement `Borrow<Self>`. You can borrow a `&[T]` from a `Vec<T>`, so `[T]` can implement `ToOwned<Owned=Vec<T>>`, as long as `T` implements `Clone`, so that we can copy the slice’s elements into the vector